In [175]:
import requests
from pprint import  pprint
import json

In [176]:
information = ['bibtex','title','author','type','group', 'tag', 'journal', 'preprint_id','URL','DOI','ISBN','ISSN','misc']
mergedDictionary= {
  "title": None,
  "user": None,
  "author": None,
  "entrytype":None,
  "group": None,
  "tag": None,
  "journal": None,
  "preprint_id": None,
  "URL": None,
  "DOI": None,
  "ISBN": None,
  "ISSN": None,
  "misc": None
}
doi="10.1007/s10494-016-9704-y"
#doi="10.2218/iclass.2021.5815"
#doi = "10.1137/140985482"
response = requests.get(f"http://api.crossref.org/works/org/works/{doi}")


In [177]:

def orderAuthors(authors):
    authorsOrdered =""
    for author in authors:
        authorsOrdered += author['family']+", "+author['given']
        if author != authors[-1]:
           authorsOrdered += " and "
    return authorsOrdered


In [178]:

response = response.json()['message']
#response = response['message']
#response = {k.lower(): v for k, v in response.items()}
pprint(response)

{'DOI': '10.1007/s10494-016-9704-y',
 'ISSN': ['1386-6184', '1573-1987'],
 'URL': 'http://dx.doi.org/10.1007/s10494-016-9704-y',
 'alternative-id': ['9704'],
 'author': [{'affiliation': [],
             'family': 'Beck',
             'given': 'Andrea D.',
             'sequence': 'first'},
            {'affiliation': [],
             'family': 'Flad',
             'given': 'David G.',
             'sequence': 'additional'},
            {'affiliation': [],
             'family': 'Tonhäuser',
             'given': 'Claudia',
             'sequence': 'additional'},
            {'affiliation': [],
             'family': 'Gassner',
             'given': 'Gregor',
             'sequence': 'additional'},
            {'affiliation': [],
             'family': 'Munz',
             'given': 'Claus-Dieter',
             'sequence': 'additional'}],
 'container-title': ['Flow, Turbulence and Combustion'],
 'content-domain': {'crossmark-restriction': False,
                    'domain': ['link.sprin

In [179]:


for i in information:
    if i == 'author':
        orderedAuthors = orderAuthors(response['author'])
        mergedDictionary["author"] = orderedAuthors
    elif i == 'type':
        mergedDictionary['entrytype'] = str(response['type'])
    elif i == 'title':
        mergedDictionary['title'] = str(response['title'][0])
    elif i in response.keys():
        mergedDictionary[i] = str(response[i])
    #else:
        #infoTest += ' '+i+'= None ,'

print(json.dumps(mergedDictionary,indent=7))



{
       "title": "On the Influence of Polynomial De-aliasing on Subgrid Scale Models",
       "user": null,
       "author": "Beck, Andrea D. and Flad, David G. and Tonh\u00e4user, Claudia and Gassner, Gregor and Munz, Claus-Dieter",
       "entrytype": "journal-article",
       "group": null,
       "tag": null,
       "journal": null,
       "preprint_id": null,
       "URL": "http://dx.doi.org/10.1007/s10494-016-9704-y",
       "DOI": "10.1007/s10494-016-9704-y",
       "ISBN": null,
       "ISSN": "['1386-6184', '1573-1987']",
       "misc": null
}


In [180]:
import dictdiffer
import os
from pyPUMA import PUMACollection, PUMAEntry
# 1 - Fetch data from PUMA (Replace if necessary or add to environment vars)
username = "jo-hollenbeck"
token = "5efe8593931e3c325f9399812b1de5da"
collection = PUMACollection.from_web_api(username, token)
# Find all duplicates and sort into data structure
duplicates = collection.gather_duplicates("doi")
record = duplicates[doi]

In [181]:
for tup in dictdiffer.diff(record[0].dict(exclude={"bibtex"}), record[1].dict(exclude={"bibtex"})):
    print(tup)
#doi="10.2218/iclass.2021.5815"
mergedPumaEntries = PUMAEntry.from_multiple_entries(duplicates[doi])
print(mergedPumaEntries)

('change', 'author', ('Beck, Andrea D. and Flad, David G. and Tonh{\\"a}user, Claudia and Gassner, Gregor and Munz, Claus-Dieter', 'Beck, A. and Flad, D. and Tonhäuser, C. and Gassner, G. and Munz, C.-D.'))
('change', ['tag', 1], ('from:katharinafuchs', 'from:simtechpuma'))
('change', ['tag', 2], ('from:simtechpuma', 'imported'))
('remove', 'tag', [(3, 'pn1')])
('change', 'journal', ('Flow, Turbulence and Combustion', 'Flow Turbulence Combustion'))
('change', 'url', ('https://doi.org/10.1007/s10494-016-9704-y', 'http://doi.org/10.1007/s10494-016-9704-y'))
('change', 'issn', ('1573-1987', None))
title='On the Influence of Polynomial De-aliasing on Subgrid Scale Models' user='simtech' author='Beck, Andrea D. and Flad, David G. and Tonh{\\"a}user, Claudia and Gassner, Gregor and Munz, Claus-Dieter' entrytype='article' tag=['imported', 'from:katharinafuchs', 'pn1', 'EXC310', 'from:simtechpuma'] journal='Flow Turbulence Combustion' preprint_id=None url='https://doi.org/10.1007/s10494-016-97

In [182]:

print(json.dumps(mergedDictionary,indent=7))

mergedPumaEntries = PUMAEntry.from_multiple_entries(duplicates[doi])
#
for key in mergedDictionary.keys():
    if mergedDictionary[key] == 'authors':
        if mergedPumaEntries.author.len() > mergedDictionary['authors'].len():
            mergedDictionary['authors'] = mergedPumaEntries.author
    if mergedDictionary[key] is None:
        switch_dict = {
            "title": mergedPumaEntries.title,
            "user": mergedPumaEntries.user,
            "author": mergedPumaEntries.author,
            "entrytype":mergedPumaEntries.entrytype,
            'tag': mergedPumaEntries.tag,
            #"group": mergedPumaEntries.group,
            "journal": mergedPumaEntries.journal,
            "preprint_id": mergedPumaEntries.preprint_id,
            "URL": mergedPumaEntries.url,
            "DOI": mergedPumaEntries.doi,
            "ISBN": mergedPumaEntries.isbn,
            "ISSN": mergedPumaEntries.issn,
            "misc": mergedPumaEntries.misc,
        }
        mergedDictionary[key] = switch_dict.get(key)



{
       "title": "On the Influence of Polynomial De-aliasing on Subgrid Scale Models",
       "user": null,
       "author": "Beck, Andrea D. and Flad, David G. and Tonh\u00e4user, Claudia and Gassner, Gregor and Munz, Claus-Dieter",
       "entrytype": "journal-article",
       "group": null,
       "tag": null,
       "journal": null,
       "preprint_id": null,
       "URL": "http://dx.doi.org/10.1007/s10494-016-9704-y",
       "DOI": "10.1007/s10494-016-9704-y",
       "ISBN": null,
       "ISSN": "['1386-6184', '1573-1987']",
       "misc": null
}


In [183]:
print(json.dumps(mergedDictionary,indent=7))


{
       "title": "On the Influence of Polynomial De-aliasing on Subgrid Scale Models",
       "user": "simtech",
       "author": "Beck, Andrea D. and Flad, David G. and Tonh\u00e4user, Claudia and Gassner, Gregor and Munz, Claus-Dieter",
       "entrytype": "journal-article",
       "group": null,
       "tag": [
              "imported",
              "from:katharinafuchs",
              "pn1",
              "EXC310",
              "from:simtechpuma"
       ],
       "journal": "Flow Turbulence Combustion",
       "preprint_id": null,
       "URL": "http://dx.doi.org/10.1007/s10494-016-9704-y",
       "DOI": "10.1007/s10494-016-9704-y",
       "ISBN": null,
       "ISSN": "['1386-6184', '1573-1987']",
       "misc": null
}


In [184]:
def merge(doi):
    mergedDictionary= {
        "title": None,
        "user": None,
        "author": None,
        "entrytype":None,
        #"group": None,
        #"tag": None,
        "EXC": None,
        "pn":None,
        "peerReviewed": None,
        "graduateschool": None,
        "journal": None,
        "preprint_id": None,
        "URL": None,
        "DOI": None,
        "ISBN": None,
        "ISSN": None,
        "misc": None
    }
    response = requests.get(f"http://api.crossref.org/works/org/works/{doi}")
    response = response.json()['message']
    
    for i in information:
        if i == 'author':
            orderedAuthors = orderAuthors(response['author'])
            mergedDictionary["author"] = orderedAuthors
        elif i == 'type':
            mergedDictionary['entrytype'] = str(response['type'])
        elif i == 'title':
            mergedDictionary['title'] = str(response['title'][0])
        elif i in response.keys():
            mergedDictionary[i] = str(response[i])
        #else:
            #infoTest += ' '+i+'= None ,'


    mergedPumaEntries = PUMAEntry.from_multiple_entries(duplicates[doi])
    for key in mergedDictionary.keys():
        if mergedDictionary[key] == 'authors':
            if len(mergedPumaEntries.author) > len(mergedDictionary['authors']):
                mergedDictionary['authors'] = mergedPumaEntries.author
        if mergedDictionary[key] is None:
            switch_dict = {
                "title": mergedPumaEntries.title,
                "user": mergedPumaEntries.user,
                "author": mergedPumaEntries.author,
                "entrytype":mergedPumaEntries.entrytype,
                #'tag': mergedPumaEntries.tag,
                
                #"group": mergedPumaEntries.group,
                "journal": mergedPumaEntries.journal,
                "preprint_id": mergedPumaEntries.preprint_id,
                "URL": mergedPumaEntries.url,
                "DOI": mergedPumaEntries.doi,
                "ISBN": mergedPumaEntries.isbn,
                "ISSN": mergedPumaEntries.issn,
                "misc": mergedPumaEntries.misc,
            }
            for tag in mergedPumaEntries.tag:
                
                if "EXC" in tag.upper():
                    
                    if ("missing" in tag.lower()) & (mergedDictionary["EXC"] is None):
                        mergedDictionary["EXC"]= tag.upper()
                    else:
                        mergedDictionary["EXC"]= tag.upper()

                elif "pn" in tag.lower():

                    if "missing" in tag.lower():
                         if mergedDictionary["pn"] is None:
                            mergedDictionary["pn"]= tag.lower()
                    else:
                        if mergedDictionary["pn"] is None:
                            mergedDictionary["pn"]= tag[:3].lower()
                        elif "missing" in mergedDictionary["pn"]:
                            mergedDictionary["pn"]= tag[:3].lower()

                elif "peerreviewed" in tag.lower():
                    mergedDictionary["peerReviewed"]= tag
                elif "graduateschool" in tag.lower():
                    mergedDictionary["graduateschool"]= tag
            mergedDictionary[key] = switch_dict.get(key)
    return mergedDictionary

In [185]:
import pandas as pd
data = []
doiList = []
for dup in duplicates:
    if (dup != '10.17192/bfdm.2022.1.8412') & (dup != '10.18419/opus-9726') & (dup != '10.1109/med.2015.7158818'):
        doiList.append(dup)
print(len(doiList))
for doi in doiList:
    data.append(merge(doi))
df = pd.DataFrame(data)

79


In [186]:
df.to_csv("name.csv", index=False, sep=",")

#excxxx pn(-)Zahl #peerreviewed  #graduate school